In [1]:
#이 부분은 처음 한번만 실행하면 됌.
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
 
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
 
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

     |████████████████████████████████| 958 kB 5.5 MB/s 
     |████████████████████████████████| 138 kB 52.7 MB/s 
     |████████████████████████████████| 356 kB 37.5 MB/s 
     |████████████████████████████████| 3.6 MB 36.5 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 58 kB 4.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.down

In [2]:
#해당 url로 이동
url = "https://www.naver.com/" 
driver.get(url)
 
update = driver.find_element_by_css_selector('#NM_TS_ROLLING_WRAP > div > div')
print(update.text)

이슈
코로나바이러스감염증-19 현황


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """


In [3]:
import re
import requests
from bs4 import BeautifulSoup

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
cine21_url = 'http://www.cine21.com/rank/person/'
res = requests.get(cine21_url)
print(res.text) # get으로 할 경우 배우 리스트가 응답으로 반환되지 않음(사이트마다 다름)

In [5]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2022-01'
post_data['gender'] = 'all'
post_data['page'] = 1

res = requests.post(cine21_url, data=post_data)

In [6]:
res.text

'\t\t\t<ul class="people_list">\r\n\t\t\t\t<li class="people_li">\r\n\t\t\t\t\t<a href="/db/person/info/?person_id=78487"><img src="https://image.cine21.com/resize/cine21/still/2017/1207/15_06_46__5a28da76c2e01[X145,145].jpg" alt="" class="people_thumb" target="_blank" /></a>\r\n\t\t\t\t\t<div class="name"><a href="/db/person/info/?person_id=78487">강하늘(2편)</a></div>\r\n\t\t\t\t\t<ul class="num_info">\r\n\t\t\t\t\t\t<li><span class="tit">흥행지수</span><strong>79,392</strong></li>\r\n\t\t\t\t\t\t<!--\r\n\t\t\t\t\t\t<li><a href="#" class="btn_graph"><span class="ico"></span><span>흥행성적<br />그래프로 보기</span></a></li>\r\n\t\t\t\t\t\t-->\r\n\t\t\t\t\t</ul>\r\n\t\t\t\t\t<!-- 영화포스터는 최대 5개까지만 -->\r\n\t\t\t\t\t<ul class="mov_list">\r\n\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t<li>\r\n\t\t\t\t\t\t\t\t<a href="/movie/info/?movie_id=56540">\r\n\t\t\t\t\t\t\t\t\t<img src="https://image.cine21.com/resize/cine21/poster/2022/0127/56540_61f1fcfdd84ce[X85,120].jpg" alt="" class="thumb"  target="_blank" />\r\n\t\t\t\t\t\t\

In [7]:
soup = BeautifulSoup(res.text, 'html.parser')

In [77]:
actors_info_list = list()
actors_0 = soup.select('li.people_li')
for index, actors in enumerate(actors_0) :
    actor = actors.select_one('div.name')
    actor_info_dict = dict()
    actor_name = re.sub('\(\w*\)', '', actor.text)
    actor_info_dict['이름'] = actor_name
    
    actor_link = 'http://www.cine21.com' + actor.select_one('a')['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    
    for actor_detail in actor_details:
        actor_detail_key = actor_detail.select_one('span.tit').text
        actor_detail_value = re.sub('<span.*>.*</span>', '', str(actor_detail))
        actor_detail_value = re.sub('<.*?>', '', actor_detail_value)
        actor_info_dict[actor_detail_key] = actor_detail_value
   
   # actor_add_info_key = ['흥행지수', '출연작품', '랭킹']
    actor_num_info_val = actors.select_one('ul.num_info').text.strip()[4:]
    actor_mov_list_val =  actors.select_one('ul.mov_list').text.strip()
    actor_mov_list_val_list = actor_mov_list_val.split('\n\n\n\n\n\n')
    actor_grade_val = actors.select_one('span.grade').text
    actor_info_dict['흥행지수'] = actor_num_info_val
    actor_info_dict['출연작품'] = actor_mov_list_val_list
    actor_info_dict['랭킹'] = actor_grade_val
    
    actors_info_list.append(actor_info_dict)
    ####actors_info_list.append(~~~)
actors_info_list

[{'다른 이름': '김하늘',
  '랭킹': '1',
  '생년월일': '1990-02-21',
  '성별': '남',
  '신장/체중': '181cm, 70kg',
  '이름': '강하늘',
  '직업': '배우',
  '출연작품': ['해적: 도깨비 깃발', '해피 뉴 이어'],
  '학교': '중앙대학교 연극학과',
  '홈페이지': '\nhttp://weibo.com/galpos3?is_hot=1\n',
  '흥행지수': '79,392'},
 {'랭킹': '2',
  '생년월일': '1987-02-22',
  '성별': '여',
  '신장/체중': '170cm',
  '이름': '한효주',
  '직업': '배우',
  '출연작품': ['해적: 도깨비 깃발'],
  '취미': '영화감상',
  '학교': '동국대학교 연극영화',
  '홈페이지': '\nhttps://www.facebook.com/hhj.official\n',
  '흥행지수': '67,602'},
 {'랭킹': '3',
  '생년월일': '1985-07-14',
  '성별': '남',
  '소속사': '킹콩엔터테인먼트',
  '신장/체중': '190cm',
  '이름': '이광수',
  '직업': '배우',
  '출연작품': ['해적: 도깨비 깃발', '해피 뉴 이어'],
  '홈페이지': '\nhttps://twitter.com/masijacoke85\nhttps://www.instagram.com/masijacoke850714/\n',
  '흥행지수': '59,677'},
 {'다른 이름': 'Kwon Sang Woo',
  '랭킹': '4',
  '생년월일': '1976-08-05',
  '성별': '남',
  '소속사': '벨액터스 엔터테인먼트',
  '신장/체중': '183cm, 72kg',
  '이름': '권상우',
  '직업': '배우',
  '출연작품': ['해적: 도깨비 깃발'],
  '취미': '수영, 헬스, 복싱',
  '특기': '농구',
  '학교': '한남대학교 

In [ ]:
#흥행지수, 출연영화(리스트로 만들기), 랭킹 추가
#1-10페이지까지 확장
